In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## LOAD DATASET

In [3]:
train = pd.read_csv("./data/train.tsv", sep = "\t", index_col = "PhraseId")
train["Phrase(original)"] = train["Phrase"]
print(train.shape)
train.head(10)

(156060, 4)


,SentenceId,Phrase,Sentiment,Phrase(original)
PhraseId,,,,
1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...
2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...
3,1,A series,2,A series
4,1,A,2,A
5,1,series,2,series
6,1,of escapades demonstrating the adage that what...,2,of escapades demonstrating the adage that what...
7,1,of,2,of
8,1,escapades demonstrating the adage that what is...,2,escapades demonstrating the adage that what is...
9,1,escapades,2,escapades


In [4]:
test = pd.read_csv("./data/test.tsv", sep = "\t", index_col = "PhraseId")
test["Phrase(original)"] = train["Phrase"]
print(test.shape)
test.head(10)


(66292, 3)


,SentenceId,Phrase,Phrase(original)
PhraseId,,,
156061,8545,An intermittently pleasing but mostly routine ...,NaN
156062,8545,An intermittently pleasing but mostly routine ...,NaN
156063,8545,An,NaN
156064,8545,intermittently pleasing but mostly routine effort,NaN
156065,8545,intermittently pleasing but mostly routine,NaN
156066,8545,intermittently pleasing but,NaN
156067,8545,intermittently pleasing,NaN
156068,8545,intermittently,NaN
156069,8545,pleasing,NaN


## PREPROCESSING

## Cleantext -- 1

In [5]:
#영어 단수 복수 같은거 전부 원본으로 교체
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

stemmer.stem("disappointments") #disappointment


'disappoint'

In [6]:
def stem_phrase(phrase):
    words = phrase.split(" ")
    stemmed_words = []

    for word in words:
        stemmed_word = stemmer.stem(word)
        stemmed_words.append(stemmed_word)

    stemmed_phrase = " ".join(stemmed_words)

    return stemmed_phrase

# Train
tqdm.pandas(desc="Stemming... (train)")

train["Phrase"] = train["Phrase"].progress_apply(stem_phrase)

print(train.shape)
train.head()

# Test
tqdm.pandas(desc = "Stemming... (test)")

test["Phrase"] = test["Phrase"].progress_apply(stem_phrase)

print(test.shape)
test.head()


Stemming... (train):   0%|          | 0/156060 [00:00<?, ?it/s]

Stemming... (train):   1%|          | 1248/156060 [00:00<00:12, 12473.11it/s]

Stemming... (train):   2%|▏         | 2441/156060 [00:00<00:12, 12301.81it/s]

Stemming... (train):   2%|▏         | 3482/156060 [00:00<00:13, 11660.87it/s]

Stemming... (train):   3%|▎         | 4228/156060 [00:00<00:16, 9258.36it/s] 

...

Stemming... (test):  98%|█████████▊| 64918/66292 [00:05<00:00, 10288.19it/s]

Stemming... (test):  99%|█████████▉| 65955/66292 [00:05<00:00, 10141.56it/s]

Stemming... (test): 100%|██████████| 66292/66292 [00:05<00:00, 11063.15it/s]

(66292, 3)


,SentenceId,Phrase,Phrase(original)
PhraseId,,,
156061,8545,an intermitt pleas but most routin effort .,NaN
156062,8545,an intermitt pleas but most routin effort,NaN
156063,8545,an,NaN
156064,8545,intermitt pleas but most routin effort,NaN
156065,8545,intermitt pleas but most routin,NaN


# Cleantext -- 2

In [7]:
def clean_text(phrase):
    phrase = phrase.replace("n't", "not")
    phrase = phrase.replace("hopeless", "bad")
    phrase = phrase.replace("good", "best")
    phrase = phrase.replace("excellent", "best")
    phrase = phrase.replace("funni", "fun")
    phrase = phrase.replace("funny", "fun")
    phrase = phrase.replace("littl", "little")
    phrase = phrase.replace("the movi", "movie")
    phrase = phrase.replace("veri", "very")
    phrase = phrase.replace("onli", "only")
    phrase = phrase.replace("comedi", "comedy")
    phrase = phrase.replace("veri", "very")
    phrase = phrase.replace("stori", "story")
    phrase = phrase.replace("charact", "character")
    
    #Hypothesis
    phrase = phrase.replace("not", "no")
    return phrase

train["Phrase"] = train["Phrase"].apply(clean_text)
test["Phrase"] = test["Phrase"].apply(clean_text)


## one Hot Encode Phrases

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

char_vectorizer = TfidfVectorizer(analyzer = 'char', 
                                  max_features = 10000, 
                                  ngram_range = (1,9))

char_vectorizer.fit(train["Phrase"])
%time 
#analyzer = word, character
#기타 - 피쳐로 단어수

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs


In [9]:
word_vectorizer = TfidfVectorizer(analyzer = 'word', 
                                  max_features = 30000, 
                                  ngram_range = (1,4))

word_vectorizer.fit(train["Phrase"])

#analyzer = word, character

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### fit and transform </br></br>
Fitting finds the internal parameters of a model that will be used to transform data. </br></br>

Transforming applies the parameters to data.</br></br>

You may fit a model to one set of data, and then transform it on a completely different set. </br>

In [10]:
X_train_word = word_vectorizer.transform(train["Phrase"])
print(X_train_word.shape)
X_train_char = char_vectorizer.transform(train["Phrase"])
print(X_train_char.shape)

from scipy.sparse import hstack #vstack, hstack (word, character to merge )
X_train = hstack([X_train_char, X_train_word])
print(X_train.shape)
X_train

(156060, 30000)


(156060, 10000)


(156060, 40000)


<156060x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 19680572 stored elements in COOrdinate format>

In [11]:
X_test_word = word_vectorizer.transform(test["Phrase"])
print(X_test_word.shape)
X_test_char = char_vectorizer.transform(test["Phrase"])
print(X_test_char.shape)

from scipy.sparse import hstack #vstack, hstack (word, character to merge )
X_test = hstack([X_test_char, X_test_word])
print(X_test.shape)
X_test

(66292, 30000)


(66292, 10000)


(66292, 40000)


<66292x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 7612200 stored elements in COOrdinate format>

In [12]:
y_train = train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

## Score 

* random forest classifier 하기에 column이 1000개나 되어서, 너무 오래걸림 </br></br>

#### Machine learning</br>
- Supervised: if label exist </br>
    - Tree =  DecisionTree, RandomForest(+Ensemble)</br>
    - Regression = 선형(y = ax + b, x = feature, y = label)</br></br></br>
    - cross validation to pick which one you prefer: 
    use "recent_reservations" and find the features that matters. Divide the data to trian and test and rerun the analysis !
- Unsupervised: ^ otherwise </br>
- Reinforcement: 강화학습은 너무 유니크하다? 거의 쓰이지않는다? </br></br>



In [13]:
from sklearn.linear_model import SGDClassifier

seed = 23

model = SGDClassifier(n_jobs=-1,
                      alpha = 0.00005,
                      random_state=seed)

model

SGDClassifier(alpha=5e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=23, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [14]:
#cross validation

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
# http://scikit-learn.org/stable/modules/classes.html
predictions = cross_val_predict(model, X_train, y_train, cv = 5)

score = accuracy_score(y_train, predictions)

print("Score = {0:5f}".format(score))


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Score = 0.592452


모델 정확도를 높히는 방법으로, 미리 맞춰놓은 값들을 더 잘 맞출 필요는 없다. 
그래서 못맞춘 데이터만 떼어서, 그걸 계속 튜닝하는식으로 진행하는게 훨씬 효과적이다.

In [15]:
result = pd.DataFrame({'actual': y_train, 'predict': predictions})

result['distance'] = np.abs(result['actual'] - result['predict'])
result['Phrase'] = train['Phrase']
result = result.sort_values(by = 'distance', ascending = False)

result.head(1000).to_csv("cv_results_test_5.csv")
print(result.shape)
result.head()
#틀린이유 / 중요키워드 > 데이터 클리닝 > 다시돌리기...
#위에 코이피션트 확인하러 올라감.

(156060, 4)


,actual,predict,distance,Phrase
PhraseId,,,,
145348,0,4,4,is hard a masterpiec
23806,0,4,4,"on it ici face , the new film is a subzero ver..."
11777,0,4,4,", this is the opposit of a truli magic movi ."
11778,0,4,4,this is the opposit of a truli magic movi .
66532,0,4,4,one of the saddest action hero perform ever wit


In [16]:
from sklearn.model_selection import cross_val_score, GroupKFold
# from sklearn.cross_validation import cross_val_score, GroupKFold

kfold = GroupKFold(n_splits=5)

score = cross_val_score(model, X_train, y_train, cv=kfold, groups=train["SentenceId"]).mean()

print("Score = {0:.5f}".format(score))
#57962
#58078
#58579
#58597

/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


/Users/rosekoh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Score = 0.59852


## Predictions

In [17]:
import xgboost as xgb

In [24]:
predictions = booster.predict(dtest)
print(predictions.shape)
predictions[0:10]

(66292,)


array([ 3.,  3.,  2.,  3.,  3.,  3.,  3.,  2.,  3.,  2.], dtype=float32)

In [18]:
dtrain = xgb.DMatrix(X_train, label = y_train)

In [19]:
params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'lambda': 2.186753e-03,
    'alpha': 1.286904,
    'lambda_bias': 6.191707e+00,
    'num_class': 5,
    'nthread':2,
}

booster = xgb.train(params, dtrain, num_boost_round=98)
booster

In [22]:
dtest = xgb.DMatrix(X_test.toarray())

NameError: name 'predictons' is not defined

# Submit

In [25]:
submission = pd.read_csv("./data/sampleSubmission.csv", index_col="PhraseId")

submission["Sentiment"] = predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3.0
156062,3.0
156063,2.0
156064,3.0
156065,3.0


In [ ]:
submission.to_csv("tfidf-xgboost.csv")